### **1. Напишите декоратор, реализующий проверку по заданному правилу входных аргументов функции ( @pre ) .При невыполнении условий прерывайте выполнение с выдачей заранее заданного сообщения. Декоратор должен принимать в качестве аргументов правило и сообщение.**

In [1]:
def pre(condition, message):
    def wrapper(func):
        def inner(*args):
            for x in args:
                if not condition(x):
                    return message
            return func(*args)
        return inner
    return wrapper


new_pre = pre(lambda x: x > 0, 'Некорректные данные!')


@new_pre
def nums(*args):
    return args


print(nums(1, 2))
print(nums(1, -2))

(1, 2)
Некорректные данные!


### **2. Напишите декоратор, реализующий проверку по заданному правилу выходных аргументов функции ( @post ) . Все остальное - как в примере выше.**

In [2]:
import random


def post(condition, message):
    def wrapper(func):
        def inner(*args):
            res = func(*args)
            for x in res:
                if condition(x):
                    return message, res
            return res
        return inner
    return wrapper


new_post = post(lambda x: x != 0 and x % 7 == 0, 'Некорректные данные!')


@new_post
def nums(n):
    return tuple(i * random.randint(0, 1) for i in range(n))


print(nums(10))
print(nums(11))

(0, 0, 2, 3, 4, 0, 0, 0, 8, 9)
('Некорректные данные!', (0, 0, 2, 3, 4, 5, 6, 7, 0, 0, 10))


### **3. Напишите настраиваимый декоратор, обеспечивающий мемоизацию (проверьте его на функции Аккермана или на вычислении n-го числа Фибоначчи). Мемоизатор должен хранить не более n значений, (как в очереди -- если пришло n+1 -ое) -- вы забываете 1 -ое. После выполнения функции выдавайте на печать сколько раз функция посчиталась, и сколько раз вы взяли значение из памяти.**

In [2]:
from_func_count = 0
from_cache_count = 0


def fibonacci_decorator(memoize_count):
    cache = dict()

    def wrapper(func):
        def inner(n):
            global from_func_count
            global from_cache_count

            if n not in cache.keys():
                if len(tuple(cache.items())) >= memoize_count:
                    cache.pop(tuple(cache.keys())[0])

                cache[n] = func(n)
                from_func_count += 1
            else:
                from_cache_count += 1

            return cache[n]
        return inner
    return wrapper


new_fib_dec = fibonacci_decorator(2)


@new_fib_dec
def fibonacci(n):
    if n in (1, 2):
        return 1

    return fibonacci(n - 1) + fibonacci(n - 2)


print(fibonacci(5))
print(f'Функция посчиталась {from_func_count} раз(-а)\n'
      f'Значение взято из памяти {from_cache_count} раз(-а)')


5
Функция посчиталась 5 раз(-а)
Значение взято из памяти 2 раз(-а)


### **4. Напишите декоратор определяющий среднее выполнение времени функции по результатам n запусков**

In [7]:
import time

from_func_count = 0
from_cache_count = 0
avg_time = 0


def time_decorator(repeat_number):
    def wrapper(func):
        def inner(n):
            global avg_time

            for i in range(repeat_number):
                start = time.time()
                func(n)
                avg_time += time.time() - start
            return avg_time / repeat_number
        return inner
    return wrapper


def fib_decorator(memoize_count):
    cache = dict()

    def wrapper(func):
        def inner(n):
            global from_func_count
            global from_cache_count

            if len(tuple(cache.items())) == memoize_count:
                cache.pop(tuple(cache.keys())[0])

            if n not in tuple(cache.keys()):
                cache[n] = func(n)
                from_func_count += 1
            else:
                from_cache_count += 1
            return cache[n]
        return inner
    return wrapper


new_fib_dec = fib_decorator(2)
new_time_dec = time_decorator(10)


@new_fib_dec
@new_time_dec
def fib(n):
    if n in (1, 2):
        return 1

    return fib(n - 1) + fib(n - 2)


fib(100)
print(f'Функция посчиталась {from_func_count} раз(-а)\n'
      f'Значение взято из памяти {from_cache_count} раз(-а)\n'
      f'Среднее время выполнения {avg_time} с')


Функция посчиталась 118 раз(-а)
Значение взято из памяти 1843 раз(-а)
Среднее время выполнения 0.46889543533325195 с
